In [10]:
import sys
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import tsfresh.feature_extraction.feature_calculators
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


### Of all the parameters, you may need to change only model_file, csv_file and n_jobs

In [11]:
model_file = "gene_age_predictions_14traits.cbm"
csv_file = "All_prog_ident_tt.csv"
n_jobs = 8
params = {'Target len mismatches ratio': {'quantile': [{'q': 0.1}, {'q': 0.6}],
  'fft_coefficient': [{'coeff': 0, 'attr': 'abs'}],
  'abs_energy': None},
 'Bit score': {'fft_aggregated': [{'aggtype': 'variance'},
   {'aggtype': 'centroid'}]},
 'Alignment len mismatches ratio': {'fft_coefficient': [{'coeff': 0,
    'attr': 'abs'}],
  'abs_energy': None},
 'Query len mismatches ratio': {'abs_energy': None}}
feats = ['Target len mismatches ratio__quantile__q_0.1',
 'Query len mismatches ratio_3',
 'Target len mismatches ratio__quantile__q_0.6',
 'Bit score__fft_aggregated__aggtype_"variance"',
 'Alignment len mismatches ratio__fft_coefficient__coeff_0__attr_"abs"',
 'Percent identity_7',
 'Target len mismatches ratio__fft_coefficient__coeff_0__attr_"abs"',
 'Bit score__fft_aggregated__aggtype_"centroid"',
 'Alignment len mismatches ratio__abs_energy',
 'Target len mismatches ratio__abs_energy',
 'Target len mismatches ratio_2',
 'Query len mismatches ratio_1',
 'Percent identity_10',
 'Query len mismatches ratio__abs_energy']

### Character Generation

In [12]:
df = pd.read_csv("All_prog_ident_tt.csv")
for i in range(1,11):
    s_q = f'Start position in query_{i}'
    e_q = f'End position in query_{i}'
    q_l = f'Query len_{i}'
    df[q_l] = df[e_q] - df[s_q]    
    s_t = f'Start position in target_{i}'
    e_t = f'End position in target_{i}'
    t_l = f'Target len_{i}'
    df[t_l] = df[e_t] - df[s_t]
    lens_ratio = f'Query Target lens ratio_{i}' #
    df[lens_ratio] = df[q_l] / df[t_l]
    
    m = f'Number of mismatches_{i}'
    a_l = f'Alignment length_{i}'
    tl_m_ration = f'Target len mismatches ratio_{i}'
    ql_m_ration = f'Query len mismatches ratio_{i}'
    al_m_ration = f'Alignment len mismatches ratio_{i}'
    df[tl_m_ration] = df[m] / df[t_l]
    df[ql_m_ration] = df[m] / df[q_l]
    df[al_m_ration] = df[m] / df[a_l]
df = df.replace([np.inf, -np.inf], np.nan).fillna(-999)


### Single-thread version of feature extraction

In [13]:
%%time
feature_calculators_module = sys.modules['tsfresh.feature_extraction.feature_calculators']
for col_name in params:
    for func_name in params[col_name]:
        func_params = params[col_name][func_name]
        trait_name = f'{col_name}__{func_name}'
        func = getattr(feature_calculators_module, func_name)
        if func_params is not None:
            for func_param_dict in func_params:
                trait_name_postfix = ''
                for key, val in func_param_dict.items():
                    if isinstance(val, str): val = f'"{val}"'
                    elif isinstance(val, list): val = "({})".format(', '.join(str(e) for e in val))
                    trait_name_postfix += f'__{key}_{val}'
                if func.fctype=='simple':
                    df[trait_name+trait_name_postfix] = df[list(df.filter(regex=f'{col_name}_\d+'))].apply(
                        lambda x: func(x,**func_param_dict),
                        raw=True,axis=1)                    
                else:
                    df[trait_name+trait_name_postfix] = df[list(df.filter(regex=f'{col_name}_\d+'))].apply(
                        lambda x: list(*func(x,[func_param_dict]))[1],
                        raw=True,axis=1)
        else:
            df[trait_name] = df[list(df.filter(regex=f'{col_name}_\d+'))].apply(func,raw=True,axis=1)

CPU times: user 53.1 s, sys: 243 ms, total: 53.4 s
Wall time: 53.4 s


### Multithreaded version of feature extraction

In [9]:
%%time
feature_calculators_module = sys.modules['tsfresh.feature_extraction.feature_calculators']
def extraction_ts_traits(col_name,df,params):
    res = pd.DataFrame()
    for func_name in params[col_name]:
        func_params = params[col_name][func_name]
        trait_name = f'{col_name}__{func_name}'
        func = getattr(feature_calculators_module, func_name)
        if func_params is not None:
            for func_param_dict in func_params:
                trait_name_postfix = ''
                for key, val in func_param_dict.items():
                    if isinstance(val, str): val = f'"{val}"'
                    elif isinstance(val, list): val = "({})".format(', '.join(str(e) for e in val))
                    trait_name_postfix += f'__{key}_{val}'
                if func.fctype=='simple':
                    res[trait_name+trait_name_postfix] = df[list(df.filter(regex=f'{col_name}_\d+'))].apply(
                        lambda x: func(x,**func_param_dict),
                        raw=True,axis=1)                    
                else:
                    res[trait_name+trait_name_postfix] = df[list(df.filter(regex=f'{col_name}_\d+'))].apply(
                        lambda x: list(*func(x,[func_param_dict]))[1],
                        raw=True,axis=1)
        else:
            res[trait_name] = df[list(df.filter(regex=f'{col_name}_\d+'))].apply(func,raw=True,axis=1)
    return res

processed_list = Parallel(n_jobs=n_jobs)(delayed(extraction_ts_traits)(i,df,params) for i in tqdm(params.keys()))
# need to uncomment to use the results of the multithreaded version
#df = pd.concat([df,*processed_list],axis=1)

100%|██████████| 4/4 [00:00<00:00, 4271.19it/s]


CPU times: user 391 ms, sys: 133 ms, total: 525 ms
Wall time: 53.2 s


### In target target, we keep the true labels for our classes
We solve the binary classification problem: young if age <= 12 otherwise other. In target, we keep the true labels for our classes in order to evaluate accuracy later.

In [14]:
target = df.Age.apply( lambda x: 'other' if (x<=12) else 'young' )

### Only 14 traits are needed for prediction!

In [15]:
df[feats]

,Target len mismatches ratio__quantile__q_0.1,Query len mismatches ratio_3,Target len mismatches ratio__quantile__q_0.6,"Bit score__fft_aggregated__aggtype_""variance""","Alignment len mismatches ratio__fft_coefficient__coeff_0__attr_""abs""",Percent identity_7,"Target len mismatches ratio__fft_coefficient__coeff_0__attr_""abs""","Bit score__fft_aggregated__aggtype_""centroid""",Alignment len mismatches ratio__abs_energy,Target len mismatches ratio__abs_energy,Target len mismatches ratio_2,Query len mismatches ratio_1,Percent identity_10,Query len mismatches ratio__abs_energy
0,0.030002,0.030403,0.058579,0.154165,0.475833,93.700,0.477014,0.046081,2.528372e-02,2.541265e-02,0.033634,0.027116,93.200,2.544817e-02
1,0.088949,0.101266,0.131093,0.449810,1.197402,83.500,1.206256,0.137111,1.527466e-01,1.552068e-01,0.091471,0.067089,81.000,1.581622e-01
2,0.038341,0.043689,0.084745,0.215719,0.756377,88.900,0.759025,0.077030,6.670157e-02,6.720798e-02,0.038835,0.033981,88.200,6.746819e-02
3,0.113967,0.128205,0.165107,0.326661,1.503316,80.400,1.514723,0.112439,2.360670e-01,2.394950e-01,0.115979,0.094872,80.900,2.385597e-01
4,0.072414,0.084615,0.124626,0.977100,1.191889,85.000,1.197494,0.298265,1.645983e-01,1.661485e-01,0.072797,0.069231,76.000,1.686881e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89599,0.456530,0.504505,0.538002,2.240921,4.841114,42.149,4.948759,0.856279,2.459561e+00,2.562571e+00,0.486957,0.169643,44.348,2.660376e+00
89600,-999.000000,-999.000000,-999.000000,2.883657,8990.773333,-999.000,8990.753623,1.142353,8.982009e+06,8.982009e+06,-999.000000,0.229730,-999.000,8.982009e+06
89601,0.499900,0.574007,0.546075,0.955407,4.992723,33.333,5.250930,0.338027,2.618248e+00,2.900264e+00,0.542662,0.204545,28.793,3.225888e+00
89602,-999.000000,-999.000000,-999.000000,2.877101,8990.788462,-999.000,8990.755556,1.137625,8.982009e+06,8.982009e+06,-999.000000,0.234043,-999.000,8.982009e+06


### Make a prediction
You can put "CPU" or "GPU" in task_type.

In [16]:
%%time
model = CatBoostClassifier(task_type="CPU")
model.load_model(model_file)
preds = model.predict(df[feats])
preds

CPU times: user 23.2 ms, sys: 37.8 ms, total: 61 ms
Wall time: 26.1 ms


array(['other', 'other', 'other', ..., 'young', 'young', 'young'],
      dtype=object)

### Rate accuracy

In [17]:
print("Classification report\n",classification_report(target, preds))

Classification report
               precision    recall  f1-score   support

       other       0.94      0.94      0.94     76951
       young       0.63      0.61      0.62     12653

    accuracy                           0.89     89604
   macro avg       0.78      0.78      0.78     89604
weighted avg       0.89      0.89      0.89     89604



### If necessary, you can predict not class labels, but probabilities

In [18]:
model.predict_proba(df[feats])

array([[0.68848035, 0.31151965],
       [0.68848035, 0.31151965],
       [0.68848035, 0.31151965],
       ...,
       [0.47303253, 0.52696747],
       [0.43048048, 0.56951952],
       [0.43048048, 0.56951952]])